In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import Normalizer, StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector
import torch
from torch.utils.data import Dataset
%matplotlib inline

In [2]:
# Load the data
data = pd.read_csv('./housing.csv')
print(len(data))

16512


In [3]:
data.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity,median_house_value
0,-117.61,34.13,21.0,8416.0,1386.0,4308.0,1341.0,4.4611,INLAND,164600.0
1,-117.37,33.98,52.0,201.0,44.0,130.0,24.0,2.0250,INLAND,125000.0
2,-118.34,33.89,36.0,2274.0,411.0,1232.0,423.0,5.3730,<1H OCEAN,244500.0
3,-118.92,35.13,29.0,1297.0,262.0,909.0,253.0,1.9236,INLAND,106300.0
4,-121.80,37.23,18.0,3179.0,526.0,1663.0,507.0,5.9225,<1H OCEAN,265800.0


In [4]:
# Train Test Split: Create training validation and test set, initial ratio (8:1:1)
SEED = 4
X_train_val, X_test, y_train_val, y_test = train_test_split(data.drop('median_house_value', axis=1), data.median_house_value, 
                                                            random_state=4,
                                                            test_size=0.3,
                                                           shuffle=True)

X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, 
                                                            random_state=4,
                                                            test_size=0.1,
                                                           shuffle=True)

del X_train_val
del y_train_val
print(len(X_train))
print(len(X_val))
print(len(X_test))

10402
1156
4954


In [5]:
# Visualisation
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16512 entries, 0 to 16511
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           16512 non-null  float64
 1   latitude            16512 non-null  float64
 2   housing_median_age  16512 non-null  float64
 3   total_rooms         16512 non-null  float64
 4   total_bedrooms      16344 non-null  float64
 5   population          16512 non-null  float64
 6   households          16512 non-null  float64
 7   median_income       16512 non-null  float64
 8   ocean_proximity     16512 non-null  object 
 9   median_house_value  16512 non-null  float64
dtypes: float64(9), object(1)
memory usage: 1.3+ MB


In [6]:
print(len(X_train))
print(len(X_val))
print(len(X_test))

10402
1156
4954


In [7]:
data.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,16512.00000,16512.000000,16512.000000,16512.000000,16344.000000,16512.000000,16512.000000,16512.00000,16512.000000
mean,-119.55943,35.617896,28.669331,2639.225654,538.522026,1425.169513,499.902616,3.87329,207251.882994
std,2.00348,2.134056,12.615046,2198.453859,425.256071,1147.038029,385.367776,1.91235,115687.033147
min,-124.35000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.49990,14999.000000
25%,-121.78250,33.930000,18.000000,1446.000000,295.000000,788.000000,279.000000,2.56250,120000.000000
50%,-118.49000,34.250000,29.000000,2122.000000,435.000000,1165.000000,410.000000,3.53410,179800.000000
75%,-118.00000,37.710000,37.000000,3143.000000,647.000000,1718.000000,604.000000,4.73975,264525.000000
max,-114.31000,41.950000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.00010,500001.000000


In [8]:
data.corr()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
longitude,1.000000,-0.925272,-0.112542,0.042614,0.067794,0.097204,0.052408,-0.016702,-0.049521
latitude,-0.925272,1.000000,0.015295,-0.032082,-0.063420,-0.105053,-0.066913,-0.077065,-0.139294
housing_median_age,-0.112542,0.015295,1.000000,-0.363482,-0.322493,-0.299468,-0.305436,-0.115794,0.105669
total_rooms,0.042614,-0.032082,-0.363482,1.000000,0.930445,0.855246,0.916591,0.193678,0.130938
total_bedrooms,0.067794,-0.063420,-0.322493,0.930445,1.000000,0.876054,0.978952,-0.009832,0.046566
population,0.097204,-0.105053,-0.299468,0.855246,0.876054,1.000000,0.905978,0.001281,-0.026806
households,0.052408,-0.066913,-0.305436,0.916591,0.978952,0.905978,1.000000,0.010185,0.062915
median_income,-0.016702,-0.077065,-0.115794,0.193678,-0.009832,0.001281,0.010185,1.000000,0.689606
median_house_value,-0.049521,-0.139294,0.105669,0.130938,0.046566,-0.026806,0.062915,0.689606,1.000000


In [9]:
# Imbalance in the categorical values
# sns.pairplot(data)

## Preprocessing

In [10]:
X_train.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
3958,-122.45,37.71,46.0,2559.0,506.0,1562.0,498.0,4.3846,NEAR OCEAN
8485,-117.71,33.54,15.0,2460.0,368.0,962.0,320.0,7.3878,<1H OCEAN
7116,-119.69,36.80,31.0,2576.0,458.0,1306.0,418.0,3.2813,INLAND
14244,-123.79,39.44,16.0,2017.0,423.0,1177.0,414.0,3.2171,<1H OCEAN
9503,-118.11,33.98,36.0,446.0,108.0,410.0,117.0,3.3942,<1H OCEAN


In [11]:
one_hot_imputer_transformer = ColumnTransformer([('one_hot_encoder', OneHotEncoder(), ['ocean_proximity']),
                  ('imputer', SimpleImputer(missing_values=np.nan, strategy='mean'), 
                   make_column_selector(dtype_include=np.number))], remainder='passthrough')
one_hot_imputed_data = pd.DataFrame(one_hot_imputer_transformer.fit_transform(X_train), columns=one_hot_imputer_transformer.get_feature_names_out())

# Find Categorical Columns
non_categorical_columns = []
for c in one_hot_imputed_data.columns:
    if 'one_hot_encoder' not in c:
        non_categorical_columns.append(c)
        
normalizer = make_column_transformer((StandardScaler(), non_categorical_columns), remainder='passthrough')
data_preprocessed = pd.DataFrame(normalizer.fit_transform(one_hot_imputed_data), columns=normalizer.get_feature_names_out())

In [12]:
data_preprocessed.head()

,standardscaler__imputer__longitude,standardscaler__imputer__latitude,standardscaler__imputer__housing_median_age,standardscaler__imputer__total_rooms,standardscaler__imputer__total_bedrooms,standardscaler__imputer__population,standardscaler__imputer__households,standardscaler__imputer__median_income,remainder__one_hot_encoder__ocean_proximity_<1H OCEAN,remainder__one_hot_encoder__ocean_proximity_INLAND,remainder__one_hot_encoder__ocean_proximity_ISLAND,remainder__one_hot_encoder__ocean_proximity_NEAR BAY,remainder__one_hot_encoder__ocean_proximity_NEAR OCEAN
0,-1.432291,0.966542,1.363852,-0.034077,-0.074579,0.120501,-0.002678,0.271941,0.0,0.0,0.0,0.0,1.0
1,0.938199,-0.986905,-1.087422,-0.078933,-0.402913,-0.392589,-0.465351,1.848511,1.0,0.0,0.0,0.0,0.0
2,-0.052006,0.540250,0.177752,-0.026375,-0.188782,-0.098417,-0.210621,-0.307251,0.0,1.0,0.0,0.0,0.0
3,-2.102429,1.776966,-1.008349,-0.279650,-0.272055,-0.208732,-0.221018,-0.340954,1.0,0.0,0.0,0.0,0.0
4,0.738158,-0.780786,0.573118,-0.991448,-1.021513,-0.864632,-0.993006,-0.247983,1.0,0.0,0.0,0.0,0.0


In [14]:
data_preprocessed

,standardscaler__imputer__longitude,standardscaler__imputer__latitude,standardscaler__imputer__housing_median_age,standardscaler__imputer__total_rooms,standardscaler__imputer__total_bedrooms,standardscaler__imputer__population,standardscaler__imputer__households,standardscaler__imputer__median_income,remainder__one_hot_encoder__ocean_proximity_<1H OCEAN,remainder__one_hot_encoder__ocean_proximity_INLAND,remainder__one_hot_encoder__ocean_proximity_ISLAND,remainder__one_hot_encoder__ocean_proximity_NEAR BAY,remainder__one_hot_encoder__ocean_proximity_NEAR OCEAN
0,-1.432291,0.966542,1.363852,-0.034077,-0.074579,0.120501,-0.002678,0.271941,0.0,0.0,0.0,0.0,1.0
1,0.938199,-0.986905,-1.087422,-0.078933,-0.402913,-0.392589,-0.465351,1.848511,1.0,0.0,0.0,0.0,0.0
2,-0.052006,0.540250,0.177752,-0.026375,-0.188782,-0.098417,-0.210621,-0.307251,0.0,1.0,0.0,0.0,0.0
3,-2.102429,1.776966,-1.008349,-0.279650,-0.272055,-0.208732,-0.221018,-0.340954,1.0,0.0,0.0,0.0,0.0
4,0.738158,-0.780786,0.573118,-0.991448,-1.021513,-0.864632,-0.993006,-0.247983,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10397,1.153243,-1.155548,-1.008349,0.166640,0.127656,-0.495207,0.101294,-0.916629,0.0,0.0,0.0,0.0,1.0
10398,0.908193,-0.916637,-1.087422,-0.966981,-1.009617,-0.992050,-0.995606,0.628129,1.0,0.0,0.0,0.0,0.0
10399,-1.002202,1.973716,-1.166495,-0.501662,-0.638457,-0.646569,-0.618709,-0.525426,0.0,1.0,0.0,0.0,0.0
10400,-1.142231,0.774477,-0.059468,-0.804323,-0.805003,-0.826151,-0.748673,-0.807016,1.0,0.0,0.0,0.0,0.0


## Datasets

In [15]:
class CDataset(Dataset):
    # Nothing much here, essentially converting everything to tensor
    def __init__(self, X: pd.DataFrame, y: pd.DataFrame):
        self.X = torch.tensor(X.to_numpy(), dtype=torch.float32)
        self.y = torch.tensor(y.to_numpy(), dtype=torch.float32)
        
    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [16]:
# Validation Dataset: need to preprocess everything
# Call *.transform() not .fit_transform() here
X_val_temp = pd.DataFrame(one_hot_imputer_transformer.transform(X_val), columns=one_hot_imputer_transformer.get_feature_names_out())
X_val_preprocessed = pd.DataFrame(normalizer.transform(X_val_temp), columns= normalizer.get_feature_names_out())

In [17]:
# TODO

train_dataset = CDataset(data_preprocessed, y_train)
val_dataset = CDataset(X_val_preprocessed, y_val)

# Neural Network

In [18]:
from torch import nn
from torch.utils.data import DataLoader

In [19]:
class NN(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
          nn.Linear(13, 100),
          nn.Tanh(),
          nn.Linear(100, 70),
          nn.ReLU(),
            nn.Linear(70, 20),
          nn.ReLU(),
            nn.Linear(20, 1)
#           nn.ReLU(),
#             nn.Linear(156*2, 312),
#           nn.ReLU(),
#           nn.Linear(312, 13),
#             nn.ReLU(),
#           nn.Linear(13, 1)
        )


    def forward(self, x):
        return self.layers(x).flatten()

In [20]:
# Test the model
# dump_model = NN()
# dump_model(torch.ones((10,13))).shape

In [21]:
def train(model, dataloader, optimizer, loss_fn):
    size = len(dataloader.dataset)
    training_loss = 0
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        training_loss += loss.item()
    return training_loss / len(dataloader)


def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss = 0
    r2 = 0
    mse = 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            r2 += r2_score(pred, y)
            mse = mean_squared_error(pred, y)

    return {'Test Loss': test_loss / len(dataloader),
           'R2': r2 / len(dataloader),
           'RMSE': np.sqrt(mse/ len(dataloader))}

In [22]:
generator = torch.Generator(device='cpu')

In [23]:
from torch.utils.data import Subset

In [24]:
def subset_of_indices(train_dataset, val_dataset, train_perc=0.7, val_perc=0.4):
    """
    Return a subset of indices, randomly sampled from the train and validation
    dataset. Train_perc and Val_perc determine the size of the sample between (0,1)
    """
    train_indices = np.random.choice(np.array(range(len(train_dataset))), 
                     size= int(len(train_dataset)*train_perc), 
                     replace=False)
    val_indices = np.random.choice(np.array(range(len(val_dataset))), 
                     size= int(len(val_dataset)*val_perc), 
                     replace=False)
    return train_indices, val_indices

In [25]:
# Checking that the size of indices is correct
# train_indices, val_indices = subset_of_indices(train_dataset, val_dataset)
# assert len(Subset(train_dataset, train_indices)) == len(train_indices)

In [26]:
# TODO

train_indices, val_indices = subset_of_indices(train_dataset, val_dataset)
train_loader = DataLoader(Subset(train_dataset, train_indices), 
                          batch_size=16, 
                          shuffle=True)
valid_loader = DataLoader(Subset(val_dataset, val_indices), 
                          batch_size=len(val_dataset))

In [27]:
# def train_and_validate(config):
#     # Get the data loaders.
#     train_indices, val_indices = subset_of_indices(train_dataset, val_dataset)
#     train_loader = DataLoader(Subset(train_dataset, train_indices), 
#                               batch_size=config["batch_size"], 
#                               shuffle=True, num_workers=8)
#     valid_loader = DataLoader(Subset(val_dataset, val_indices), 
#                               batch_size=len(val_dataset), num_workers=8)

#     device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#     # Initialize the model.
#     model = NN().to(device)
#     # model = CustomNet(
#     #     config['first_conv_out'], config['first_fc_out']
#     # ).to(device)

#     # Loss Function
#     criterion = nn.MSELoss()
#     # Optimizer
#     optimizer = torch.optim.Adam(model.parameters(),
#                                  lr=config['lr'], 
#                                  weight_decay=config['weight_decay'])

#     # Start the training.
#     for epoch in range(50):
#         print(f"[INFO]: Epoch {epoch+1} of {100}")
#         train_epoch_loss = train(
#             model, train_loader, optimizer, criterion,
#         )
#         valid_epoch_loss, _ = test(
#             valid_loader, model, criterion)

#         print(f"Training loss: {train_epoch_loss:.3f}")
#         print(f"Validation loss: {valid_epoch_loss:.3f}")
#         print('-'*50)

#         # with tune.checkpoint_dir(epoch) as checkpoint_dir:
#         #     path = os.path.join(checkpoint_dir, 'checkpoint')
#         #     torch.save((model.state_dict(), optimizer.state_dict()), path)
#         tune.report(
#             train_loss=train_epoch_loss,
#             val_loss=valid_epoch_loss
#         )

In [130]:
def train_and_validate(config):
    # Get the data loaders.
#     train_loader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True)
#     valid_loader = DataLoader(val_dataset, batch_size=len(val_dataset))
    train_indices, val_indices = subset_of_indices(train_dataset, val_dataset)
    train_loader = DataLoader(Subset(train_dataset, train_indices), 
                              batch_size=16, 
                              shuffle=True)
    valid_loader = DataLoader(Subset(val_dataset, val_indices), 
                              batch_size=len(val_dataset))

    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    # Initialize the model.
    model = NN().to(device)
    # model = CustomNet(
    #     config['first_conv_out'], config['first_fc_out']
    # ).to(device)

    # Loss Function
    criterion = nn.MSELoss()
    # Optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'], weight_decay=config['weight_decay'])

    # Start the training.
    epochs = 100
    for epoch in range(epochs):
        print(f"[INFO]: Epoch {epoch+1} of {epochs}")
        train_epoch_loss = train(
            model, train_loader, optimizer, criterion,
        )
        val_metrics = test(
            valid_loader, model, criterion)

        print(f"Training loss: {train_epoch_loss:.3f}")
        print(f"Validation loss: {val_metrics['Test Loss']:.3f}")
        print('-'*50)

        # with tune.checkpoint_dir(epoch) as checkpoint_dir:
        #     path = os.path.join(checkpoint_dir, 'checkpoint')
        #     torch.save((model.state_dict(), optimizer.state_dict()), path)
        tune.report(
            train_loss=train_epoch_loss,
            val_loss=val_metrics['Test Loss'],
            val_rmse=val_metrics['RMSE']
        )

In [131]:
def run_search():
    # Define the parameter search configuration.
    config = {
        # "first_conv_out":
        #     tune.sample_from(lambda _: 2 ** np.random.randint(4, 8)),
        # "first_fc_out":
        #     tune.sample_from(lambda _: 2 ** np.random.randint(4, 8)),
        # "lr": tune.loguniform(1e-4, 1e-1),
        "lr": tune.loguniform(6e-4, 9e-4),
        "weight_decay": tune.loguniform(1e-3, 2e-3),
        "batch_size": 16
        # "batch_size": tune.choice([4, 8, 16, 32]),
    }
    # Scheduler to stop bad performing trails.
    scheduler = ASHAScheduler(
        metric="val_loss",
        mode="min",
        max_t=80,
        grace_period=40,
        reduction_factor=2)
    # Reporter to show on command line/output window
    reporter = CLIReporter(
        metric_columns=["train_loss", "val_loss"])
    # Start run/search
    result = tune.run(
        train_and_validate,
        resources_per_trial={"cpu": 4, "gpu": 0},
        config=config,
        num_samples=40,
        scheduler=scheduler,
        local_dir='raytune_results_mseloss',
        # keep_checkpoints_num=1,
        # checkpoint_score_attr='min-validation_loss',
        progress_reporter=reporter
    )
    # Extract the best trial run from the search.
    best_trial = result.get_best_trial('val_loss', 'min', 'last')
    print(f"Best trial config: {best_trial.config}")
    print(f"Best trial final validation loss: {best_trial.last_result['val_loss']}")
    # print(f"Best trial final validation acc: {best_trial.last_result['accuracy']}")


In [132]:
run_search()

== Status ==
Current time: 2022-11-23 15:52:02 (running for 00:00:00.63)
Memory usage on this node: 11.8/15.7 GiB 
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 80.000: None | Iter 40.000: None
Resources requested: 4.0/8 CPUs, 0/0 GPUs, 0.0/2.68 GiB heap, 0.0/1.34 GiB objects
Result logdir: C:\Users\stefa\VisualCodeProjects\Neural_Networks_064\raytune_results_mseloss\train_and_validate_2022-11-23_15-52-01
Number of trials: 16/40 (15 PENDING, 1 RUNNING)
+--------------------------------+----------+-----------------+-------------+----------------+
| Trial name                     | status   | loc             |          lr |   weight_decay |
|--------------------------------+----------+-----------------+-------------+----------------|
| train_and_validate_c50f9_00000 | RUNNING  | 127.0.0.1:48044 | 0.000638158 |     0.00107215 |
| train_and_validate_c50f9_00001 | PENDING  |                 | 0.000859806 |     0.00175944 |
| train_and_validate_c50f9_00002 | PENDING  |                 | 

Trial name,date,done,episodes_total,experiment_id,hostname,iterations_since_restore,node_ip,pid,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,train_loss,training_iteration,trial_id,val_loss,val_rmse,warmup_time
train_and_validate_c50f9_00000,2022-11-23_15-52-56,True,,4f473e8034b543478757e0925ee04e2d,DESKTOP-TLQ0TER,40,127.0.0.1,48044,50.1601,1.25575,50.1601,1669218776,0,,3.40334e+09,40,c50f9_00000,3.17736e+09,56368.1,0.00636697
train_and_validate_c50f9_00001,2022-11-23_15-53-41,True,,9cbd2b0ac7c842688d2b73f60e0084fb,DESKTOP-TLQ0TER,80,127.0.0.1,35520,91.2667,1.243,91.2667,1669218821,0,,2.89004e+09,80,c50f9_00001,2.87261e+09,53596.7,0.00651765
train_and_validate_c50f9_00002,2022-11-23_15-53-45,True,,4f473e8034b543478757e0925ee04e2d,DESKTOP-TLQ0TER,40,127.0.0.1,48044,48.491,1.19644,48.491,1669218825,0,,3.32248e+09,40,c50f9_00002,3.39875e+09,58298.8,0.00636697
train_and_validate_c50f9_00003,2022-11-23_15-55-16,True,,9cbd2b0ac7c842688d2b73f60e0084fb,DESKTOP-TLQ0TER,80,127.0.0.1,35520,94.6711,1.25849,94.6711,1669218916,0,,2.90904e+09,80,c50f9_00003,2.93037e+09,54132.9,0.00651765
train_and_validate_c50f9_00004,2022-11-23_15-54-32,True,,4f473e8034b543478757e0925ee04e2d,DESKTOP-TLQ0TER,40,127.0.0.1,48044,46.4638,1.12248,46.4638,1669218872,0,,3.51863e+09,40,c50f9_00004,3.62208e+09,60183.7,0.00636697
train_and_validate_c50f9_00005,2022-11-23_15-55-19,True,,4f473e8034b543478757e0925ee04e2d,DESKTOP-TLQ0TER,40,127.0.0.1,48044,47.6703,1.15549,47.6703,1669218919,0,,3.45994e+09,40,c50f9_00005,3.48434e+09,59028.3,0.00636697
train_and_validate_c50f9_00006,2022-11-23_15-56-04,True,,9cbd2b0ac7c842688d2b73f60e0084fb,DESKTOP-TLQ0TER,40,127.0.0.1,35520,47.5653,1.20214,47.5653,1669218964,0,,3.51295e+09,40,c50f9_00006,4.1415e+09,64354.5,0.00651765
train_and_validate_c50f9_00007,2022-11-23_15-56-07,True,,4f473e8034b543478757e0925ee04e2d,DESKTOP-TLQ0TER,40,127.0.0.1,48044,47.7495,1.21637,47.7495,1669218967,0,,3.36141e+09,40,c50f9_00007,4.11986e+09,64186.2,0.00636697
train_and_validate_c50f9_00008,2022-11-23_15-57-39,True,,9cbd2b0ac7c842688d2b73f60e0084fb,DESKTOP-TLQ0TER,80,127.0.0.1,35520,94.9221,1.22803,94.9221,1669219059,0,,3.09399e+09,80,c50f9_00008,2.92325e+09,54067.1,0.00651765
train_and_validate_c50f9_00009,2022-11-23_15-56-54,True,,4f473e8034b543478757e0925ee04e2d,DESKTOP-TLQ0TER,40,127.0.0.1,48044,47.1091,1.14027,47.1091,1669219014,0,,3.32583e+09,40,c50f9_00009,3.56606e+09,59716.5,0.00636697


(train_and_validate pid=48044) Training loss: 55893457515.789
(train_and_validate pid=48044) Validation loss: 50688761856.000
(train_and_validate pid=48044) --------------------------------------------------
(train_and_validate pid=48044) [INFO]: Epoch 2 of 100
(train_and_validate pid=48044) Training loss: 42219560479.754
(train_and_validate pid=48044) Validation loss: 27641026560.000
(train_and_validate pid=48044) --------------------------------------------------
== Status ==
Current time: 2022-11-23 15:52:07 (running for 00:00:05.73)
Memory usage on this node: 12.1/15.7 GiB 
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 80.000: None | Iter 40.000: None
Resources requested: 8.0/8 CPUs, 0/0 GPUs, 0.0/2.68 GiB heap, 0.0/1.34 GiB objects
Result logdir: C:\Users\stefa\VisualCodeProjects\Neural_Networks_064\raytune_results_mseloss\train_and_validate_2022-11-23_15-52-01
Number of trials: 18/40 (16 PENDING, 2 RUNNING)
+--------------------------------+----------+-----------------+------

(train_and_validate pid=48044) Training loss: 7793869095.579
(train_and_validate pid=48044) Validation loss: 6774185472.000
(train_and_validate pid=48044) --------------------------------------------------
(train_and_validate pid=48044) [INFO]: Epoch 6 of 100
(train_and_validate pid=35520) Training loss: 6157689023.579
(train_and_validate pid=35520) Validation loss: 4774338048.000
(train_and_validate pid=35520) --------------------------------------------------
(train_and_validate pid=35520) [INFO]: Epoch 6 of 100
(train_and_validate pid=35520) Training loss: 5318667001.825
(train_and_validate pid=35520) Validation loss: 4235025408.000
(train_and_validate pid=35520) --------------------------------------------------
(train_and_validate pid=35520) [INFO]: Epoch 7 of 100
(train_and_validate pid=48044) Training loss: 6495372925.193
(train_and_validate pid=48044) Validation loss: 5693136896.000
(train_and_validate pid=48044) --------------------------------------------------
(train_and_val

(train_and_validate pid=48044) Training loss: 4398515852.211
(train_and_validate pid=48044) Validation loss: 3979100672.000
(train_and_validate pid=48044) --------------------------------------------------
(train_and_validate pid=48044) [INFO]: Epoch 13 of 100
(train_and_validate pid=35520) Training loss: 3893440786.386
(train_and_validate pid=35520) Validation loss: 3429124096.000
(train_and_validate pid=35520) --------------------------------------------------
(train_and_validate pid=35520) [INFO]: Epoch 16 of 100
== Status ==
Current time: 2022-11-23 15:52:23 (running for 00:00:21.76)
Memory usage on this node: 12.2/15.7 GiB 
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 80.000: None | Iter 40.000: None
Resources requested: 8.0/8 CPUs, 0/0 GPUs, 0.0/2.68 GiB heap, 0.0/1.34 GiB objects
Result logdir: C:\Users\stefa\VisualCodeProjects\Neural_Networks_064\raytune_results_mseloss\train_and_validate_2022-11-23_15-52-01
Number of trials: 18/40 (16 PENDING, 2 RUNNING)
+----------------

...

(train_and_validate pid=48044) Training loss: 3694760477.754
(train_and_validate pid=48044) Validation loss: 4418275328.000
(train_and_validate pid=48044) --------------------------------------------------
(train_and_validate pid=48044) [INFO]: Epoch 23 of 100
(train_and_validate pid=35520) Training loss: 3132357860.456
(train_and_validate pid=35520) Validation loss: 3330738688.000
(train_and_validate pid=35520) --------------------------------------------------
(train_and_validate pid=35520) [INFO]: Epoch 64 of 100
(train_and_validate pid=48044) Training loss: 3669445461.295
(train_and_validate pid=48044) Validation loss: 4389455872.000
(train_and_validate pid=48044) --------------------------------------------------
(train_and_validate pid=48044) [INFO]: Epoch 24 of 100
(train_and_validate pid=35520) Training loss: 3124616775.377
(train_and_validate pid=35520) Validation loss: 3354922496.000
(train_and_validate pid=35520) --------------------------------------------------
(train_and_

(train_and_validate pid=35520) Training loss: 3076405356.039
(train_and_validate pid=35520) Validation loss: 3279533312.000
(train_and_validate pid=35520) --------------------------------------------------
(train_and_validate pid=35520) [INFO]: Epoch 71 of 100
(train_and_validate pid=48044) Training loss: 3521072950.596
(train_and_validate pid=48044) Validation loss: 4220483328.000
(train_and_validate pid=48044) --------------------------------------------------
(train_and_validate pid=48044) [INFO]: Epoch 31 of 100
== Status ==
Current time: 2022-11-23 16:14:07 (running for 00:22:05.99)
Memory usage on this node: 12.1/15.7 GiB 
Using AsyncHyperBand: num_stopped=38
Bracket: Iter 80.000: -3106686464.0 | Iter 40.000: -3566061824.0
Resources requested: 8.0/8 CPUs, 0/0 GPUs, 0.0/2.68 GiB heap, 0.0/1.34 GiB objects
Result logdir: C:\Users\stefa\VisualCodeProjects\Neural_Networks_064\raytune_results_mseloss\train_and_validate_2022-11-23_15-52-01
Number of trials: 40/40 (2 RUNNING, 38 TERMINA

(train_and_validate pid=35520) Training loss: 3039427459.930
(train_and_validate pid=35520) Validation loss: 3288076800.000
(train_and_validate pid=35520) --------------------------------------------------
(train_and_validate pid=48044) Training loss: 3412163121.404
(train_and_validate pid=48044) Validation loss: 4100555264.000
(train_and_validate pid=48044) --------------------------------------------------
(train_and_validate pid=48044) [INFO]: Epoch 37 of 100
(train_and_validate pid=35520) [INFO]: Epoch 77 of 100
(train_and_validate pid=48044) Training loss: 3393148002.456
(train_and_validate pid=48044) Validation loss: 4079412480.000
(train_and_validate pid=48044) --------------------------------------------------
(train_and_validate pid=35520) Training loss: 3041770351.228
(train_and_validate pid=35520) Validation loss: 3271326720.000
(train_and_validate pid=35520) --------------------------------------------------
(train_and_validate pid=35520) [INFO]: Epoch 78 of 100
(train_and_

2022-11-23 16:14:19,109	INFO tune.py:777 -- Total run time: 1337.51 seconds (1337.29 seconds for the tuning loop).


(train_and_validate pid=35520) Training loss: 3015026551.386
(train_and_validate pid=35520) Validation loss: 3307280640.000
(train_and_validate pid=35520) --------------------------------------------------
== Status ==
Current time: 2022-11-23 16:14:18 (running for 00:22:17.32)
Memory usage on this node: 11.9/15.7 GiB 
Using AsyncHyperBand: num_stopped=40
Bracket: Iter 80.000: -3145961984.0 | Iter 40.000: -3594069632.0
Resources requested: 0/8 CPUs, 0/0 GPUs, 0.0/2.68 GiB heap, 0.0/1.34 GiB objects
Result logdir: C:\Users\stefa\VisualCodeProjects\Neural_Networks_064\raytune_results_mseloss\train_and_validate_2022-11-23_15-52-01
Number of trials: 40/40 (40 TERMINATED)
+--------------------------------+------------+-----------------+-------------+----------------+--------------+-------------+
| Trial name                     | status     | loc             |          lr |   weight_decay |   train_loss |    val_loss |
|--------------------------------+------------+-----------------+-------

In [33]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    training_loss = 0
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        training_loss += loss.item()
    return {'Training Loss': training_loss / len(dataloader)}


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader)
    test_loss = 0
    r2 = 0
    mse = 0
    model.eval()
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            r2 += r2_score(pred, y)
            mse = mean_squared_error(pred, y)
    
    return {'Test Loss': test_loss / len(dataloader),
           'R2': r2 / len(dataloader),
           'MSE': mse/ len(dataloader)}

In [57]:
def train_model(epochs=100):
    model = NN()


    # learning_rate = 9e-2
    batch_size = 16

    loss_fn = nn.MSELoss()

    # HuberLoss
    # optimizer = torch.optim.Adam(model.parameters(), lr=0.0014695, weight_decay=0.0014969)
    # MSELoss
    optimizer = torch.optim.Adam(model.parameters(), lr=0.00073378, weight_decay=0.0010347)


    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=len(val_dataset))

    training_losses = []
    test_losses = []
    r2 = []
    mse = []

#     epochs = 100
    for t in range(epochs):
        training_res = train_loop(train_loader, model, loss_fn, optimizer)
        test_res = test_loop(val_loader, model, loss_fn)
        print(f'Epoch: {t+1}, {str(training_res)}, {str(test_res)}')

        training_losses.append(training_res['Training Loss'])
        test_losses.append(test_res['Test Loss'])
        mse.append(test_res['MSE'])
        r2.append(test_res['R2'])
    return model

In [58]:
# Evaluate the model
# Validation Dataset: need to preprocess everything
# Call *.transform() not .fit_transform() here
X_test_temp = pd.DataFrame(one_hot_imputer_transformer.transform(X_test), 
                          columns=one_hot_imputer_transformer.get_feature_names_out())
X_test_preprocessed = pd.DataFrame(normalizer.transform(X_test_temp), 
                                  columns= normalizer.get_feature_names_out())

In [61]:
models = []
for i in range(3):
    models.append(train_model(100))

Epoch: 1, {'Training Loss': 50810766125.22273}, {'Test Loss': 32743051264.0, 'R2': -10413.955877929482, 'MSE': 32743051264.0}
Epoch: 2, {'Training Loss': 17700767063.69278}, {'Test Loss': 9522797568.0, 'R2': -3.313540990254187, 'MSE': 9522797568.0}
Epoch: 3, {'Training Loss': 7899227811.981566}, {'Test Loss': 6646400512.0, 'R2': -0.12419471967229256, 'MSE': 6646400512.0}
Epoch: 4, {'Training Loss': 5911748118.218126}, {'Test Loss': 5453843456.0, 'R2': 0.2357623392831375, 'MSE': 5453843456.0}
Epoch: 5, {'Training Loss': 5073106558.230414}, {'Test Loss': 4962675200.0, 'R2': 0.37836336281736094, 'MSE': 4962675712.0}
Epoch: 6, {'Training Loss': 4733644373.2350235}, {'Test Loss': 4709222912.0, 'R2': 0.44466703738616997, 'MSE': 4709222400.0}
Epoch: 7, {'Training Loss': 4523288962.850999}, {'Test Loss': 4571072512.0, 'R2': 0.472957467969359, 'MSE': 4571072512.0}
Epoch: 8, {'Training Loss': 4384245151.066052}, {'Test Loss': 4478422528.0, 'R2': 0.49708523056722165, 'MSE': 4478422528.0}
Epoch: 9

Epoch: 67, {'Training Loss': 2970099576.8233485}, {'Test Loss': 3383670272.0, 'R2': 0.6755466302429631, 'MSE': 3383670272.0}
Epoch: 68, {'Training Loss': 2962835416.331797}, {'Test Loss': 3392141056.0, 'R2': 0.6820189184598893, 'MSE': 3392141056.0}
Epoch: 69, {'Training Loss': 2957350002.53149}, {'Test Loss': 3351716608.0, 'R2': 0.6633336353379473, 'MSE': 3351716608.0}
Epoch: 70, {'Training Loss': 2953395099.428571}, {'Test Loss': 3347319552.0, 'R2': 0.6718109187748593, 'MSE': 3347319552.0}
Epoch: 71, {'Training Loss': 2940684795.625192}, {'Test Loss': 3336945408.0, 'R2': 0.6803259606943528, 'MSE': 3336945664.0}
Epoch: 72, {'Training Loss': 2934909923.34255}, {'Test Loss': 3340526592.0, 'R2': 0.6769527270639136, 'MSE': 3340526848.0}
Epoch: 73, {'Training Loss': 2927934734.894009}, {'Test Loss': 3370463744.0, 'R2': 0.6657927744499705, 'MSE': 3370464000.0}
Epoch: 74, {'Training Loss': 2925856045.271889}, {'Test Loss': 3330550272.0, 'R2': 0.6796643527035604, 'MSE': 3330550272.0}
Epoch: 75

Epoch: 34, {'Training Loss': 3285759279.631336}, {'Test Loss': 3618906368.0, 'R2': 0.6312488500262321, 'MSE': 3618906112.0}
Epoch: 35, {'Training Loss': 3268785242.248848}, {'Test Loss': 3622074368.0, 'R2': 0.6338703098029306, 'MSE': 3622074368.0}
Epoch: 36, {'Training Loss': 3256938297.21659}, {'Test Loss': 3589434880.0, 'R2': 0.6500357393185271, 'MSE': 3589434880.0}
Epoch: 37, {'Training Loss': 3246342472.1597543}, {'Test Loss': 3590642688.0, 'R2': 0.6528913416357249, 'MSE': 3590642688.0}
Epoch: 38, {'Training Loss': 3227238006.562212}, {'Test Loss': 3558525440.0, 'R2': 0.6513822092904045, 'MSE': 3558525696.0}
Epoch: 39, {'Training Loss': 3215594826.02765}, {'Test Loss': 3542485760.0, 'R2': 0.6534042025279716, 'MSE': 3542486016.0}
Epoch: 40, {'Training Loss': 3204495819.158218}, {'Test Loss': 3550661888.0, 'R2': 0.6549514173066266, 'MSE': 3550662144.0}
Epoch: 41, {'Training Loss': 3187910395.870968}, {'Test Loss': 3503531008.0, 'R2': 0.6497796872696017, 'MSE': 3503531008.0}
Epoch: 42

Epoch: 100, {'Training Loss': 2788072563.24424}, {'Test Loss': 3285463296.0, 'R2': 0.696547301436798, 'MSE': 3285463040.0}
Epoch: 1, {'Training Loss': 51708200303.28725}, {'Test Loss': 36162146304.0, 'R2': -17539.641713145375, 'MSE': 36162146304.0}
Epoch: 2, {'Training Loss': 20764659899.57604}, {'Test Loss': 10750444544.0, 'R2': -6.577283275869145, 'MSE': 10750444544.0}
Epoch: 3, {'Training Loss': 8620486506.961597}, {'Test Loss': 7049597952.0, 'R2': -0.25647393783964856, 'MSE': 7049597952.0}
Epoch: 4, {'Training Loss': 6219433054.967742}, {'Test Loss': 5642083840.0, 'R2': 0.13956881581923586, 'MSE': 5642083840.0}
Epoch: 5, {'Training Loss': 5211348690.482335}, {'Test Loss': 5045348864.0, 'R2': 0.34629354556223957, 'MSE': 5045349376.0}
Epoch: 6, {'Training Loss': 4781074278.439324}, {'Test Loss': 4767977984.0, 'R2': 0.4250120684588866, 'MSE': 4767977984.0}
Epoch: 7, {'Training Loss': 4539358754.310292}, {'Test Loss': 4598439936.0, 'R2': 0.47458110253254393, 'MSE': 4598440448.0}
Epoch:

Epoch: 67, {'Training Loss': 2954468273.4500766}, {'Test Loss': 3355569152.0, 'R2': 0.6724152352288726, 'MSE': 3355568896.0}
Epoch: 68, {'Training Loss': 2946149706.420891}, {'Test Loss': 3383489280.0, 'R2': 0.6828738331748196, 'MSE': 3383489536.0}
Epoch: 69, {'Training Loss': 2942081465.658986}, {'Test Loss': 3328317184.0, 'R2': 0.6777800573009181, 'MSE': 3328317184.0}
Epoch: 70, {'Training Loss': 2939336251.3794165}, {'Test Loss': 3356086272.0, 'R2': 0.6884537524566484, 'MSE': 3356086272.0}
Epoch: 71, {'Training Loss': 2928482253.984639}, {'Test Loss': 3330838528.0, 'R2': 0.683780077658064, 'MSE': 3330838528.0}
Epoch: 72, {'Training Loss': 2923097380.0061445}, {'Test Loss': 3309753088.0, 'R2': 0.6877276038319919, 'MSE': 3309753088.0}
Epoch: 73, {'Training Loss': 2913159978.764977}, {'Test Loss': 3361761024.0, 'R2': 0.6898678940479486, 'MSE': 3361761024.0}
Epoch: 74, {'Training Loss': 2920129991.963134}, {'Test Loss': 3313308160.0, 'R2': 0.6823317171251136, 'MSE': 3313308160.0}
Epoch:

In [62]:
# Not tested
mse_acc = []
r2_acc = []
rmse_acc = []
for model in models:
    test_dataset = CDataset(X_test_preprocessed, y_test)
    # test_loader = DataLoader(test_dataset, batch_size=len(test_dataset))
    mseloss = nn.MSELoss()

    x,y = test_dataset[:]
    with torch.no_grad():
        model.eval()
        y_predict = model(x)
        mse_acc.append(mseloss(y_predict, y).item())
        r2_acc.append(r2_score(y_predict, y))
#         rmse_acc.append(np.sqrt(y_predict, y))
print(f'MSE: {np.mean(mse_acc)}')
print(f'R2 Score: {np.mean(r2_acc)}')
print(f'RMSE: {np.sqrt(mse_acc).mean()}')
print(f'MSE: {np.std(mse_acc)}')
print(f'R2 Score: {np.std(r2_acc)}')
print(f'RMSE: {np.sqrt(mse_acc).std()}')

MSE: 2983008512.0
R2 Score: 0.7151645267621686
RMSE: 54616.753774034274
MSE: 14941908.74407934
R2 Score: 0.00825087771358447
RMSE: 136.81807816377324


In [50]:
mse_acc

[3010422016.0, 3010422016.0, 3010422016.0]

mse_acc